In [1]:
#The variables which are commented are to use the CPU instead of the GPU
#The os variable set in -1 will disable the GPU

#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


#The threads variables are used to set the number of threads to be used when CPU test are needed

# Number of threads
#os.environ['TF_NUM_INTEROP_THREADS'] = '32'
#os.environ['TF_NUM_INTRAOP_THREADS'] = '32'

2023-08-08 17:20:10.973565: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-08 17:20:11.929475: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9899750526362080611
xla_global_id: -1
]


2023-08-08 17:20:13.006094: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-08-08 17:20:13.006142: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: ip-172-31-19-119
2023-08-08 17:20:13.006151: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: ip-172-31-19-119
2023-08-08 17:20:13.006234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.86.5
2023-08-08 17:20:13.006264: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.86.5
2023-08-08 17:20:13.006273: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.86.5


In [2]:
import time
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Read the datasets
train_data = pd.read_csv('UNSW_NB15_training-set.csv')
test_data = pd.read_csv('UNSW_NB15_testing-set.csv')

# 'label' is the binary target column
label_column = 'label'

# All other columns are feature columns
feature_columns = train_data.columns.tolist()
feature_columns.remove(label_column)

X_train = train_data[feature_columns]
y_train = train_data[label_column]
X_test = test_data[feature_columns]
y_test = test_data[label_column]

# Label Encoding for the binary target variable
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# One-hot encoding for the categorical features
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Ensure the test data has the same columns as the training data
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Scale feature data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Wrap the model creation and training within a MirroredStrategy scope
#The line of code 'GPUS = ["GPU:0"]' and 'strategy = tf.distribute.MirroredStrategy(GPUS)' should be commented when CPU is selected

GPUS = ["GPU:0"]
strategy = tf.distribute.MirroredStrategy(GPUS)

# Building the Neural Network model within the MirroredStrategy scope
# The strategy line code 'with strategy.scope():' should be commented when the CPU is used for the data processing
# The tabular space should be also eliminated to avoid errors

with strategy.scope():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification output layer

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model using fit() as usual (no need to change this part)
    start_time_binary = time.time()
    model.fit(X_train, y_train, epochs=10, verbose=1)
    end_time_binary = time.time()

# Calculate training time for binary classification
training_time_binary = end_time_binary - start_time_binary
print("Binary Classification - Training Time:", training_time_binary, "seconds")

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.2f' % (accuracy * 100))


Epoch 1/10
644/644 [==============================] - 4s 5ms/step - loss: 0.3081 - accuracy: 0.9927 - val_loss: 0.0523 - val_accuracy: 0.9986
Epoch 2/10
644/644 [==============================] - 3s 5ms/step - loss: 0.0301 - accuracy: 0.9999 - val_loss: 0.0302 - val_accuracy: 0.9990
Epoch 3/10
644/644 [==============================] - 3s 5ms/step - loss: 0.0185 - accuracy: 1.0000 - val_loss: 0.0221 - val_accuracy: 0.9991
Epoch 4/10
644/644 [==============================] - 3s 5ms/step - loss: 0.0131 - accuracy: 1.0000 - val_loss: 0.0172 - val_accuracy: 0.9990
Epoch 5/10
644/644 [==============================] - 3s 4ms/step - loss: 0.0108 - accuracy: 1.0000 - val_loss: 0.0150 - val_accuracy: 0.9989
Epoch 6/10
644/644 [==============================] - 3s 5ms/step - loss: 0.0089 - accuracy: 1.0000 - val_loss: 0.0137 - val_accuracy: 0.9990
Epoch 7/10
644/644 [==============================] - 3s 5ms/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 0.0137 - val_accuracy: 0.9990
Epoch 

In [3]:
from sklearn.metrics import f1_score, recall_score, precision_score

# Make predictions on the test data
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)

# Calculate metrics
f1 = f1_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)

# Print metrics
print('Test F1 Score: %.2f' % f1)
print('Test Recall: %.2f' % recall)
print('Test Precision: %.2f' % precision)

5480/5480 [==============================] - 5s 886us/step
Test F1 Score: 1.00
Test Recall: 1.00
Test Precision: 1.00


In [4]:
# Calculate metrics
f1 = f1_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)

# Print metrics
print('Test F1 Score: %.2f' % f1)
print('Test Recall: %.2f' % recall)
print('Test Precision: %.2f' % precision)

Test F1 Score: 1.00
Test Recall: 1.00
Test Precision: 1.00
